In [2]:
%run base_functions.ipynb

In [3]:
import traceback
import math
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,explained_variance_score


In [4]:
default_val = None
source_file='my_portfolio.txt'
target_file='my_portfolio.csv'

In [5]:
# file format must be SYMBOL,QTY,INVESTMENT,AVG
my_port = load_portfolio_file(source_file)  
#my_port = my_port.append(load_portfolio_file('my_portfolio.txt'))
my_port_index_symbol = my_port.set_index('SYMBOL')
my_port

,SYMBOL,DATE,INVESTMENT,REAL,QTY,AVG,DAYS,CATEGORY
0,TANLA,2020-12-21,"9,548.67",0.00,15,636.58,164,< 1 Year
1,TANLA,2020-12-08,0.00,"1,677.23",0,NaN,177,< 1 Year
2,PNB,2020-12-08,"7,815.00",0.00,200,39.08,177,< 1 Year
3,WIPRO,2020-12-23,"3,795.95",9.25,10,379.59,162,< 1 Year
4,UNITECH,2013-12-05,"9,260.20",0.00,400,23.15,2737,> 5 Year
5,DCBBANK,2012-05-14,"6,554.64",0.00,150,43.70,3307,> 5 Year
6,NHPC,2020-12-04,"8,902.80",533.92,400,22.26,181,< 1 Year
7,NHPC,2013-09-06,"15,200.20",800.00,600,25.33,2827,> 5 Year
8,RAYMOND,2020-12-21,"5,297.78",0.00,15,353.19,164,< 1 Year
9,SJVN,2020-12-11,"7,643.70",499.50,300,25.48,174,< 1 Year


In [10]:
get_latest_data(['AMARAJABAT'],latest=True)

loading ticker info https://www.nseindia.com/api/quote-equity AMARAJABAT


dict_values([{'i_symbol': 'AMARAJABAT', 'i_companyName': 'Amara Raja Batteries Limited', 'i_industry': 'AUTO ANCILLARIES', 'i_activeSeries_0': 'EQ', 'i_isFNOSec': True, 'i_isCASec': False, 'i_isSLBSec': True, 'i_isDebtSec': False, 'i_isSuspended': False, 'i_isETFSec': False, 'i_isDelisted': False, 'i_isin': 'INE885A01032', 'i_isTop10': False, 'i_identifier': 'AMARAJABATEQN', 'm_series': 'EQ', 'm_symbol': 'AMARAJABAT', 'm_isin': 'INE885A01032', 'm_status': 'Listed', 'm_listingDate': '05-Sep-2003', 'm_industry': 'BATTERIES - AUTOMOBILE', 'm_lastUpdateTime': '03-Jun-2021 10:41:14', 'm_pdSectorPe': 31.07, 'm_pdSymbolPe': 19.58, 'm_pdSectorInd': 'NIFTY 500                                         ', 'securityInfo_boardStatus': 'Main', 'securityInfo_tradingStatus': 'Active', 'securityInfo_tradingSegment': 'Normal Market', 'securityInfo_sessionNo': '-', 'securityInfo_slb': 'Yes', 'securityInfo_classOfShare': 'Equity', 'securityInfo_derivatives': 'Yes', 'securityInfo_surveillance': '-', 'securi

In [ ]:
prediction_columns = ['f_o', 'f_h','f_l', 'f_c']
reference_columns = ['o', 'h', 'l', 'c', 'v']
columns = ['o','h','l','c']
range_key = 'Data Range'
# Price Band, Delivery %, Company Revenue (Quaterly), P/L (Quaterly),

In [ ]:
my_port_index_symbol

In [ ]:
%%capture
market_data = load_price_history(my_port['SYMBOL'],365*6)

In [ ]:
def time_filter(target_df,time_key):
    return target_df[target_df.apply(time_dims[time_key],axis=1)].reset_index()

def ticker_filter(target_df,ticker):
    return target_df[target_df['SYMBOL']==ticker].reset_index()


In [ ]:
merge_market_data = pd.DataFrame();
merge_market_data = merge_market_data.append([x.data for x in market_data],ignore_index=True, sort=False)

In [ ]:
summary_data_all = pd.DataFrame();

for time_key in time_dims:
    time_filter_data = time_filter(merge_market_data,time_key)
    time_filter_data[range_key] = time_key
    summary_data_all = summary_data_all.append(time_filter_data)
    
    
summary_data_all.groupby(['SYMBOL',range_key])['c'].agg(metrics)


In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
p = sns.lineplot(x='time', y='c',ax=ax,data=merge_market_data, hue='SYMBOL', color='r')
#sns.lineplot(x='time', y='o',ax=ax,data=merge_market_data, hue='SYMBOL', color='g')
plt.xticks(rotation=30)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# cufflink and plotly for intercting with charts
# Make interactive points here to the statistical data (Above 1 week, 1 month, 3 month etc) 
# Draw my portolio avg dotted line here 

In [ ]:
chart_rows = math.ceil(len(time_dims) / 2)

for data in market_data:
    chart_count = 0
    row_count = 0
    fig, axes = plt.subplots(nrows=chart_rows, ncols=2,figsize=(10,chart_rows * 4))
    display(HTML('<center><b>'+data.name))
    for key in time_dims:
        p = sns.regplot(x='t', y='o', data=time_filter(data.data,key),scatter_kws={'s':2},line_kws={'linewidth':.75}, ax=axes[row_count,chart_count % 2])
        #p.fig.tight_layout()
        #p.fig.subplots_adjust(top=0.95)
        #p.fig.suptitle(data.name+' - '+key, fontsize=12)
        #plt.xticks(rotation=30)
        p.set_title(key)
        plt.setp(p.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        chart_count = chart_count + 1
        if(chart_count!=0 and chart_count % 2 == 0):
            row_count = row_count +1
    
    fig.tight_layout()        
    plt.show()
    
        
# instead of col, it should use have used radio or dropdown of ticker
# Charts should be rendered for selected Ticker/Symbol 
# create chart based on price segment , [0-40] , [40 - 100], so that chart have clear trend 

In [ ]:
def predict_lm_values(lm_df,prediction_date,y_column,time_frame,debug=False):
    len_df = len(lm_df)
    last_index = len_df - 1
    
    # copy values for future
    for c in columns:
        lm_df['f_'+c] = lm_df[c].shift(-prediction_shift)
    
    if(debug):
        display(lm_df.tail(prediction_shift+1))
        
    drop_index = np.arange(start=len_df - prediction_shift, stop=len_df)
    # prepare data
    lm_df = lm_df.drop(drop_index)
    
    if(debug):
        display(lm_df.tail(2))
        
    # prepare columns
    X = lm_df[reference_columns]
    y = lm_df[y_column]
    
    
    #train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
    
    # preparing model
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    display(lm.coef_)
    
    # testing model 
    y_predicted = lm.predict(X_test)

    plt.scatter(y_test,y_predicted,s=2);
    plt.xlabel('Y test ( True Values)')
    plt.ylabel('Predicted value')
    plt.show()
    
    # Creating prediction data
    display(X_future[reference_columns])
    predict_val = lm.predict(X_future[reference_columns])

    final_prediction_data.append({'SYMBOL':ticker,
                               'prediction_date':prediction_date,
                               'value': predict_val[0]
                                ,'time_frame' : time_frame
                                ,'last_value':X_future[y_column[2]].values[0]
                               ,'column' : y_column
                                ,'MAE':mean_absolute_error(y_test,y_predicted)
                                ,'MSE':mean_squared_error(y_test,y_predicted)
                                ,'RMSE': np.sqrt(mean_squared_error(y_test,y_predicted))
                                ,'explained_variance_score': explained_variance_score(y_test,y_predicted)
                               })

    

In [ ]:
final_prediction_data = []

In [ ]:
prediction_date = datetime.fromtimestamp( market_data[0].data.tail(1)['t'].values[0] ) + timedelta(days=prediction_shift)
while prediction_date.weekday() > 4:
    prediction_date = prediction_date + timedelta(days=1)
prediction_date

In [ ]:
time_dims

In [ ]:
# time dimension analysis with 1 week, 1 month , 3 month, 6 month , 1 year, 5 year
# Ticker for the linear model prediction
#ticker = 'ZEEL'
selected_time_dims = ['1 week','Overall']
for key_time in time_dims:
    if 'month' not in key_time:
        continue
        
    prediction_shift = 1

    time_filtered_data = time_filter(merge_market_data,key_time);
    print('Data point after time filter ',len(time_filtered_data))
    for ticker in my_port['SYMBOL']:   
        display(HTML('<center><b>PREDICTING Values for ticker ' + ticker))
        lm_df = ticker_filter(time_filtered_data,ticker)

        X_future = lm_df.iloc[[len(lm_df)-1]]

        lm_df.groupby('SYMBOL')['c'].agg(metrics)
        for y_column in prediction_columns:
            display(HTML('<center><b>PREDICTING Values for ticker:' + ticker + ' day: ' + str(prediction_shift) + ' column: '+y_column))
            try:
                predict_lm_values(lm_df,prediction_date,y_column,key_time)
            except Exception as e:
                print('error creating model - ',ticker,y_column)
                traceback.print_exc()
                #raise e


In [ ]:
predicted_values = pd.DataFrame(final_prediction_data).set_index(['SYMBOL','prediction_date','column','value']).drop_duplicates()
predicted_values = predicted_values.reset_index()
predicted_values['change'] = predicted_values['value'] - predicted_values['last_value']

predicted_values['p_Date'] = prediction_date
# updated
update_portfolio_file(predicted_values,'pred_'+target_file,False)
display(predicted_values)

In [ ]:
predicted_closing_value = predicted_values[predicted_values['column']=='f_c']
predicted_portfolio =my_port.merge(predicted_closing_value)
predicted_portfolio['predicted_val'] = predicted_portfolio['QTY'] * predicted_portfolio['change']
display(predicted_portfolio)

update_portfolio_file(predicted_portfolio,'port_'+target_file,False)


In [ ]:
predicted_portfolio.columns

In [ ]:
for key in time_dims:
    print(key)
    display(predicted_portfolio[predicted_portfolio['time_frame']==key].groupby('SYMBOL')['predicted_val'].agg(['sum']))
    display(predicted_portfolio[predicted_portfolio['time_frame']==key]['predicted_val'].agg(['sum']))